# Reproduction of Adult dataset experiments

In this notebook we reproduce the results from Table 2 of the DECAF paper. We compare various methods for generating debiased data using the DECAF model against synthetic data generated using benchmark models GAN, WGAN-GP and FairGAN. As described in the paper we run all experiments (as implemented in this notebook) 10 times and avarage the results.

In [1]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

from data import load_adult, preprocess_adult
from metrics import DP, FTU
from train import train_decaf, train_fairgan, train_vanilla_gan, train_wgan_gp


## Loading data

In [2]:
dataset = load_adult()
dataset.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Preprocess the data next in order to make it suitable for training models on.

In [3]:
dataset = preprocess_adult(dataset)
dataset.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.301370,0.833333,0.043338,0.000000,0.800000,0.333333,0.615385,0.6,0.0,1.0,0.02174,0.0,0.397959,0.0,1.0
1,0.452055,0.166667,0.047277,0.000000,0.800000,0.000000,0.307692,0.4,0.0,1.0,0.00000,0.0,0.122449,0.0,1.0
2,0.287671,0.000000,0.137244,0.200000,0.533333,0.166667,0.461538,0.6,0.0,1.0,0.00000,0.0,0.397959,0.0,1.0
3,0.493151,0.000000,0.150212,0.133333,0.400000,0.000000,0.461538,0.4,1.0,1.0,0.00000,0.0,0.397959,0.0,1.0
4,0.150685,0.000000,0.220703,0.000000,0.800000,0.000000,0.384615,0.0,1.0,0.0,0.00000,0.0,0.397959,0.3,1.0


Split the dataset into train and test folds. Test fold size is 2000.

In [4]:
# Split data into train and testing sets
dataset_train, dataset_test = train_test_split(dataset, test_size=2000,
                                               stratify=dataset['income'])

print('Size of train set:', len(dataset_train))
print('Size of test set:', len(dataset_test))

Size of train set: 28162
Size of test set: 2000


### Defining the DAG

We need to define a DAG which captures the biases of the dataset. As described in the DECAF paper normally a causal discovery algorithm is used. In this notebook we simply copy the DAG which as described in the Zhang et al. paper which is the one also used in the DECAF paper.

In [5]:
# Define DAG for Adult dataset
dag = [
    # Edges from race
    ['race', 'occupation'],
    ['race', 'income'],
    ['race', 'hours-per-week'],
    ['race', 'education'],
    ['race', 'marital-status'],

    # Edges from age
    ['age', 'occupation'],
    ['age', 'hours-per-week'],
    ['age', 'income'],
    ['age', 'workclass'],
    ['age', 'marital-status'],
    ['age', 'education'],
    ['age', 'relationship'],
    
    # Edges from sex
    ['sex', 'occupation'],
    ['sex', 'marital-status'],
    ['sex', 'income'],
    ['sex', 'workclass'],
    ['sex', 'education'],
    ['sex', 'relationship'],
    
    # Edges from native country
    ['native-country', 'marital-status'],
    ['native-country', 'hours-per-week'],
    ['native-country', 'education'],
    ['native-country', 'workclass'],
    ['native-country', 'income'],
    ['native-country', 'relationship'],
    
    # Edges from marital status
    ['marital-status', 'occupation'],
    ['marital-status', 'hours-per-week'],
    ['marital-status', 'income'],
    ['marital-status', 'workclass'],
    ['marital-status', 'relationship'],
    ['marital-status', 'education'],
    
    # Edges from education
    ['education', 'occupation'],
    ['education', 'hours-per-week'],
    ['education', 'income'],
    ['education', 'workclass'],
    ['education', 'relationship'],
    
    # All remaining edges
    ['occupation', 'income'],
    ['hours-per-week', 'income'],
    ['workclass', 'income'],
    ['relationship', 'income'],
]

def dag_to_idx(df, dag):
    """Convert columns in a DAG to the corresponding indices."""

    dag_idx = []
    for edge in dag:
        dag_idx.append([df.columns.get_loc(edge[0]), df.columns.get_loc(edge[1])])

    return dag_idx

# Convert the DAG to one that can be provided to the DECAF model
dag_seed = dag_to_idx(dataset, dag)
print(dag_seed)

[[8, 6], [8, 14], [8, 12], [8, 3], [8, 5], [0, 6], [0, 12], [0, 14], [0, 1], [0, 5], [0, 3], [0, 7], [9, 6], [9, 5], [9, 14], [9, 1], [9, 3], [9, 7], [13, 5], [13, 12], [13, 3], [13, 1], [13, 14], [13, 7], [5, 6], [5, 12], [5, 14], [5, 1], [5, 7], [5, 3], [3, 6], [3, 12], [3, 14], [3, 1], [3, 7], [6, 14], [12, 14], [1, 14], [7, 14]]


It's also necessary to define edges we want to remove from the DAG in order to meet the various fairness criteria described in the paper.

In [6]:
def create_bias_dict(df, edge_map):
    """
    Convert the given edge tuples to a bias dict used for generating
    debiased synthetic data.
    """
    bias_dict = {}
    for key, val in edge_map.items():
        bias_dict[df.columns.get_loc(key)] = [df.columns.get_loc(f) for f in val]
    
    return bias_dict

# Bias dictionary to satisfy FTU
bias_dict_ftu = create_bias_dict(dataset, {'income': ['sex']})
print('Bias dict FTU:', bias_dict_ftu)

# Bias dictionary to satisfy DP
bias_dict_dp = create_bias_dict(dataset, {'income': [
    'occupation', 'hours-per-week', 'marital-status', 'education', 'sex',
    'workclass', 'relationship']})
print('Bias dict DP:', bias_dict_dp)

# Bias dictionary to satisfy CF
bias_dict_cf = create_bias_dict(dataset, {'income': [
    'marital-status', 'sex']})
print('Bias dict CF:', bias_dict_cf)

Bias dict FTU: {14: [9]}
Bias dict DP: {14: [6, 12, 5, 3, 9, 1, 7]}
Bias dict CF: {14: [5, 9]}


## Experiments

We have loaded and preprocessed the data and we are ready to run the experiments. For each experiment we train a generative model, sample synthetic data from the trained model and then obtain metrics by training and evaluating a downstream multi-layer perceptron using the test fold we generated in the previous section. We use the MLP model from `sklearn` with default parameters which matches the settings described in Appendix D of the paper.

In [7]:
def eval_model(dataset_train, dataset_test):
    """Helper function that prints evaluation metrics."""

    X_train, y_train = dataset_train.drop(columns=['income']), dataset_train['income']
    X_test, y_test = dataset_test.drop(columns=['income']), dataset_test['income']

    clf = MLPClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    auroc = roc_auc_score(y_test, y_pred)
    dp = DP(clf, X_test)
    ftu = FTU(clf, X_test)

    return {'precision': precision, 'recall': recall, 'auroc': auroc,
            'dp': dp, 'ftu': ftu}

### Original dataset

As a benchmark we want to first train the downstream model on the original dataset.

In [8]:
eval_model(dataset_train, dataset_test)

{'precision': 0.8734977862112587,
 'recall': 0.9194407456724367,
 'auroc': 0.7589171599848128,
 'dp': 0.187192618718257,
 'ftu': 0.035499999999999976}

In the following sections we train various models in order to reproduce the results from Table 2 of the DECAF paper.

### GAN

In [9]:
synth_data = train_vanilla_gan(dataset_train)
synth_data.head()

2022-01-27 14:44:21.123162: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-27 14:44:21.210210: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
  2%|▏         | 1/50 [00:09<07:47,  9.55s/it]

0 [D loss: 0.000225, acc.: 100.00%] [G loss: 29.921358]
generated_data


  4%|▍         | 2/50 [00:17<06:52,  8.59s/it]

1 [D loss: 0.000011, acc.: 100.00%] [G loss: 53.890396]


  6%|▌         | 3/50 [00:26<06:49,  8.70s/it]

...

48 [D loss: 0.000000, acc.: 100.00%] [G loss: 316.359558]


100%|██████████| 50/50 [07:01<00:00,  8.43s/it]


49 [D loss: 0.000000, acc.: 100.00%] [G loss: 322.851257]


Synthetic data generation: 100%|██████████| 221/221 [00:02<00:00, 96.37it/s]


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.001865,0.333333,0.006739,0.133333,0.533333,0.000000,0.153846,0.0,0.25,0.0,-0.005399,0.007568,0.010692,0.775,1.0
1,0.001958,1.000000,0.006785,0.400000,0.200000,0.500000,0.769231,0.6,1.00,0.0,-0.005418,0.007586,0.010761,0.300,1.0
2,0.003629,0.500000,0.007598,0.933333,0.000000,0.333333,1.000000,0.2,0.50,1.0,-0.005579,0.008428,0.011905,0.500,0.0
3,-0.005068,0.166667,0.001333,0.666667,0.400000,0.833333,0.076923,0.2,1.00,1.0,-0.002291,0.006366,0.005182,0.075,0.0
4,-0.003677,0.500000,0.003354,0.266667,0.000000,0.666667,0.461538,0.4,0.50,1.0,-0.003649,0.006657,0.006434,0.200,1.0


In [10]:
eval_model(synth_data, dataset_test)

{'precision': 0.7497343251859724,
 'recall': 0.9394141145139814,
 'auroc': 0.49681549099193045,
 'dp': 0.015758018290198073,
 'ftu': 0.027999999999999914}

### WGAN-GP

In [11]:
synth_data = train_wgan_gp(dataset_train)
synth_data.head()

  2%|▏         | 1/50 [00:05<04:18,  5.28s/it]

Epoch: 0 | disc_loss: 0.7107614278793335 | gen_loss: -0.029137199744582176


  4%|▍         | 2/50 [00:08<03:14,  4.05s/it]

Epoch: 1 | disc_loss: 0.3214643895626068 | gen_loss: 0.0006649568094871938


  6%|▌         | 3/50 [00:11<02:47,  3.57s/it]

...

Epoch: 48 | disc_loss: -0.08630061894655228 | gen_loss: 0.010011428967118263


100%|██████████| 50/50 [02:27<00:00,  2.94s/it]


Epoch: 49 | disc_loss: -0.10573866218328476 | gen_loss: 0.1532163769006729


Synthetic data generation: 100%|██████████| 57/57 [00:00<00:00, 64.69it/s]


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.532502,0.000000,0.117296,0.133333,0.600000,0.000000,0.384615,0.2,0.5,0.0,0.043905,0.015043,0.410408,0.400,0.0
1,0.463003,1.000000,0.093717,0.666667,0.933333,0.833333,0.692308,0.8,0.5,1.0,0.039477,0.023471,0.378923,0.225,1.0
2,0.526523,0.500000,0.116040,0.266667,0.466667,0.333333,0.076923,0.4,0.0,0.0,0.062467,0.068638,0.400123,0.575,0.0
3,0.281195,1.000000,0.074402,0.000000,1.000000,0.000000,0.076923,0.6,1.0,0.0,0.012299,0.037618,0.311223,0.225,1.0
4,0.326141,0.166667,0.075756,0.133333,0.200000,0.500000,0.076923,0.6,0.0,0.0,0.024199,0.040427,0.323382,0.850,1.0


In [12]:
eval_model(synth_data, dataset_test)

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


{'precision': 0.7603550295857988,
 'recall': 0.855525965379494,
 'auroc': 0.5211364766656507,
 'dp': 0.13635587174308428,
 'ftu': 0.07699999999999996}

### FairGAN

In [13]:
synth_data = train_fairgan(dataset_train)
synth_data.head()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
cache/adult.npy


2022-01-27 14:54:35.908215: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)


Pretrain_Epoch:0, trainLoss:0.036737, validLoss:0.019772, validReverseLoss:0.000000
Pretrain_Epoch:1, trainLoss:0.019831, validLoss:0.019710, validReverseLoss:0.000000
Pretrain_Epoch:2, trainLoss:0.019880, validLoss:0.019720, validReverseLoss:0.000000
Pretrain_Epoch:3, trainLoss:0.019934, validLoss:0.019821, validReverseLoss:0.000000
Pretrain_Epoch:4, trainLoss:0.019971, validLoss:0.019784, validReverseLoss:0.000000
Pretrain_Epoch:5, trainLoss:0.019995, validLoss:0.019787, validReverseLoss:0.000000
Pretrain_Epoch:6, trainLoss:0.019959, validLoss:0.019801, validReverseLoss:0.000000
Pretrain_Epoch:7, trainLoss:0.019956, validLoss:0.019740, validReverseLoss:0.000000
Pretrain_Epoch:8, trainLoss:0.013213, validLoss:0.007593, validReverseLoss:0.000000
Pretrain_Epoch:9, trainLoss:0.007500, validLoss:0.007036, validReverseLoss:0.000000
Pretrain_Epoch:10, trainLoss:0.003250, validLoss:0.002439, validReverseLoss:0.000000
Pretrain_Epoch:11, trainLoss:0.002496, validLoss:0.002376, validReverseLoss

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.000000,0.369636,0.429304,0.568245,0.522931,0.404446,0.595719,0.000000,0.528495,0.0,0.000000,0.691437,0.000000,0.582464,1.0
1,0.513192,0.168188,0.500070,0.927547,0.000000,0.549738,0.565060,0.000000,0.711314,0.0,0.219476,0.433434,0.000000,0.000000,1.0
2,1.287684,0.025203,0.422490,1.111596,0.000000,0.458210,0.021143,0.000000,1.612983,0.0,0.000000,0.689238,0.000000,0.000000,1.0
3,0.000000,0.041467,0.410660,0.737548,0.000000,0.000000,0.614550,0.000000,0.712770,0.0,0.000000,0.520030,0.000000,0.229288,1.0
4,0.000000,0.175688,0.000000,0.529809,0.000000,0.218120,0.360827,0.300408,0.614899,0.0,0.277231,0.262351,0.107554,0.506268,1.0


In [14]:
eval_model(synth_data, dataset_test)

{'precision': 0.7911561001598295,
 'recall': 0.988681757656458,
 'auroc': 0.600766581639474,
 'dp': 0.00811793616182721,
 'ftu': 0.04049999999999998}

### DECAF

#### DECAF-ND

In [15]:
synth_data = train_decaf(dataset_train, dag_seed)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:120: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipp

Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
26022,0.234127,6.765107e-07,0.139488,0.177453,0.539940,1.179136e-01,0.298036,0.441479,1.063630e-11,1.0,2.539986e-06,2.790788e-14,0.541281,2.434887e-15,1.0
22685,0.350065,5.415779e-06,0.194560,0.133822,0.510079,2.698730e-02,0.514205,0.158220,2.920113e-20,0.0,1.781848e-09,2.888478e-12,0.604431,9.084453e-10,1.0
1786,0.107019,2.445179e-07,0.180215,0.237115,0.548000,2.801915e-01,0.295019,0.134571,0.000000e+00,0.0,2.228014e-13,3.178187e-08,0.350311,2.274254e-21,0.0
16922,0.586856,5.697811e-01,0.023120,0.191823,0.550194,5.259033e-03,0.189940,0.436645,6.327144e-37,1.0,1.146319e-01,3.676456e-10,0.270717,6.912130e-21,0.0
7506,0.419983,5.354655e-03,0.131415,0.865792,0.541115,3.556299e-08,0.743885,0.452908,2.516105e-19,1.0,4.340899e-13,2.103943e-10,0.619360,2.585483e-24,1.0


In [16]:
eval_model(synth_data, dataset_test)

{'precision': 0.8512110726643599,
 'recall': 0.8189081225033289,
 'auroc': 0.6935906074364033,
 'dp': 0.28775632,
 'ftu': 0.16600001}

#### DECAF-FTU

In [17]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_ftu)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
26022,0.060165,8.846032e-09,0.082856,0.119036,0.543828,0.506170,0.590801,0.623375,9.983974e-01,0.0,5.064419e-05,2.998538e-04,0.262476,8.257260e-01,1.0
22685,0.550186,2.414228e-03,0.181615,0.422031,0.512948,0.001664,0.087264,0.442757,9.984341e-01,1.0,4.156550e-07,2.215635e-17,0.251910,6.448075e-05,1.0
1786,0.064400,5.409464e-08,0.023076,0.104712,0.944537,0.000219,0.092931,0.424156,7.557856e-21,1.0,1.682411e-04,2.458102e-18,0.628548,1.151245e-19,0.0
16922,0.100146,2.944645e-03,0.027178,0.205918,0.543854,0.325511,0.838662,0.639171,4.482559e-31,1.0,2.821697e-11,1.048198e-16,0.391728,4.622066e-15,1.0
7506,0.406931,3.207172e-03,0.077683,0.123296,0.541885,0.009738,0.600410,0.417331,2.243062e-38,1.0,7.695793e-04,5.475842e-04,0.670021,3.697422e-16,1.0


In [18]:
eval_model(synth_data, dataset_test)

{'precision': 0.827037773359841,
 'recall': 0.8308921438082557,
 'auroc': 0.6533978791330435,
 'dp': 0.12550032,
 'ftu': 0.038999975}

#### DECAF-CF

In [19]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_cf)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
26022,0.376979,4.417319e-02,0.107903,0.090297,0.888707,0.000257,0.435016,0.445857,1.714575e-02,1.0,2.601600e-12,2.717028e-22,0.381479,2.893040e-25,0.0
22685,0.183172,3.874241e-05,0.251086,0.167791,0.551216,0.317544,0.566240,0.879151,7.999758e-21,0.0,2.024597e-14,1.635420e-18,0.286153,3.298963e-02,1.0
1786,0.228797,2.407223e-03,0.113237,0.215229,0.540730,0.002845,0.101234,0.450681,1.984082e-25,1.0,1.606261e-08,3.207013e-16,0.376381,6.985810e-17,1.0
16922,0.206276,3.136597e-08,0.067481,0.168580,0.545680,0.334844,0.283045,0.868826,0.000000e+00,0.0,1.503925e-09,9.618726e-19,0.320836,1.647236e-17,0.0
7506,0.122383,4.905114e-07,0.172160,0.244046,0.536079,0.000057,0.446719,0.315273,1.851290e-06,1.0,1.990528e-06,6.445156e-19,0.286599,6.302889e-01,0.0


In [20]:
eval_model(synth_data, dataset_test)

{'precision': 0.7640449438202247,
 'recall': 0.9507323568575233,
 'auroc': 0.5325950940914123,
 'dp': 0.003698945,
 'ftu': 0.014999986}

#### DECAF-DP

In [21]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_dp)
synth_data.head()

/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/velizarshulev/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
26022,0.219547,8.501428e-05,0.091582,0.222513,0.547538,0.322064,0.255209,0.234307,1.432031e-21,0.0,2.047043e-11,2.875638e-22,0.367917,3.855668e-06,1.0
22685,0.072794,4.437362e-09,0.135453,0.261167,0.522640,0.289771,0.316758,0.631737,2.910277e-16,0.0,3.202769e-13,1.715337e-08,0.437410,4.894960e-06,1.0
1786,0.581513,3.996564e-03,0.065573,0.264535,0.970890,0.005916,0.275685,0.426376,0.000000e+00,1.0,2.092007e-01,1.484834e-09,0.358271,5.869144e-11,1.0
16922,0.255456,1.248933e-08,0.038852,0.076800,0.539664,0.344737,0.704153,0.893334,1.005930e-04,0.0,7.739571e-08,1.118384e-15,0.527627,5.437344e-11,1.0
7506,0.071769,4.970749e-04,0.050992,0.157355,0.539611,0.281271,0.706125,0.113897,8.705056e-19,0.0,3.678382e-04,1.118564e-15,0.474221,8.006825e-02,1.0


In [22]:
eval_model(synth_data, dataset_test)

{'precision': 0.7598488936859147,
 'recall': 0.9374167776298269,
 'auroc': 0.521921240220536,
 'dp': 0.0094688535,
 'ftu': 0.036499977}